<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/fraud/Insurance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [145]:
# Elasticsearch credentials
user = "natthaphol.po"
passwords = "cnEM5CeFrG"
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
es = Elasticsearch(
    hosts=[{'host': 'elk.manageai.co.th', 'port': 443, 'scheme': 'https'}],
    basic_auth=(user, passwords)
)

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def rag_encode(text):
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
    return sentence_embeddings.squeeze().tolist()

def search_exact_match_score(query_text, index_name):
    query_vector = rag_encode(query_text)
    search_query = {
        "size": 10,
        "_source": ["icd10", "total_average_price"],  # Include only required fields
        "query": {
            "script_score": {
                "query": {
                    "match_all": {}
                },
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                    "params": {
                        "query_vector": query_vector
                    }
                }
            }
        }
    }

    try:
        response = es.search(index=index_name, body=search_query, timeout='120s')
        exact_score_matches = [
            {
                "_source": hit["_source"]
            }
            for hit in response['hits']['hits'] if hit["_score"] == 2.0
        ]

        if exact_score_matches:
            return exact_score_matches
        else:
            print("No documents found with score 2.0.")
            return None

    except Exception as e:
        print(f"Error performing search: {e}")
        return None

def search_similar_vectors(query_text, index_name):
    query_vector = rag_encode(query_text)

    search_query = {
        "size": 1000,
        "_source": [
            "simb_billing_code", "local_billing_name", "item_id", "item_name",
            "average_initial_price", "order_count", "icd10", "range"
        ],
        "query": {
            "script_score": {
                "query": {
                    "match_all": {}
                },
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                    "params": {
                        "query_vector": query_vector
                    }
                }
            }
        }
    }

    try:
        response = es.search(index=index_name, body=search_query, timeout='120s')
        results = []
        for hit in response['hits']['hits']:
            score = hit["_score"]
            if score == 2.0:
                result = {
                    "_index": hit["_index"],
                    "_id": hit["_id"],
                    "_score": score,
                    "_source": hit["_source"]
                }
                results.append(result)

        if results:
            source_data = [hit["_source"] for hit in results]
            df = pd.DataFrame(source_data)
            return df
        else:
            print("No results with score 2.0 found.")
            return None

    except Exception as e:
        print(f"Error performing search: {e}")
        return None

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

In [1]:
!pip -q install langchain langchain_huggingface langchain_community  gradio transformers elasticsearch gradio_calendar

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.1/543.1 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 

In [152]:
import os
from getpass import getpass
import requests
import os
from langchain import PromptTemplate, LLMChain, HuggingFaceHub
from huggingface_hub import InferenceClient
from elasticsearch import Elasticsearch
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
import json
import gradio as gr
from gradio_calendar import Calendar

def extract_primary_icd10(diagnosis_str):
    try:
        diagnoses = json.loads(diagnosis_str)
        if isinstance(diagnoses, list):
            for d in diagnoses:
                if isinstance(d, dict) and d.get("dx_type") == "PP":
                    icd10 = d.get("icd10")
                    if icd10:
                        # Remove the dot from the ICD-10 code
                        return icd10.replace(".", "")
    except (json.JSONDecodeError, TypeError):
        pass
    return None


In [153]:
data = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/opd_discharge_August_2024.csv')
simb = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/simblist.csv')
disease = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/diaglist.csv')
icd10_df = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/icd10 - ICD-10 v.2019.csv')
data['icd10'] = data['diagnosis'].apply(extract_primary_icd10)

In [154]:
data = data[data['claim_status'].isin(['Approve','Decline'])]
df = data[data['order_item']!='[]'][['created_at','cid','patient','illnes_type','is_follow_up','billing','order_item','total_bill_amount','insurance','diagnosis','icd10','chief_complaint']]

 # **case1:** ไม่ครอบคลุมโรคบางโรค

In [135]:
filtered_icd10 = [
    row['icd10']
    for _, row in df.iterrows()
    if any(keyword in row['response'] for keyword in ['อยู่ในกลุ่มโรค', 'เข้าข่าย'])
]
icd10_df[icd10_df['ICD-10 Code'].isin(filtered_icd10)]

,One Day Surgery : ODS,3 digit before dot,after dot,ICD-10 Code,Descriptions (EN),Descriptions (TH),illness_type,MTL,Similary,ปล.ในส่วนที่ใช้ ICD10 v.ไม่ใช่ปี2019 ตอนส่งอาจจะมีติด mapping กับประกันบ้าง แต่น้อยมาก
28,NaN,A04,9.0,A049,"Bacterial intestinal infection, unspecified",การติดเชื้อแบคทีเรียที่ลำไส้ ไม่ระบุรายละเอียด,illness,YES,NaN,NaN
805,NaN,B97,4.0,B974,Respiratory syncytial virus as the cause of di...,เรสไปราทอรี ซินซัยเทียลไวรัส ที่เป็นสาเหตุของโ...,illness,YES,NaN,NaN
2369,NaN,F32,0.0,F320,Mild depressive episode,ภาวะซึมเศร้าเล็กน้อย,illness,YES,NaN,NaN
2400,NaN,F41,9.0,F419,"Anxiety disorder, unspecified",โรควิตกกังวล ไม่ระบุรายละเอียด,illness,YES,NaN,NaN
2452,NaN,F52,4.0,F524,Premature ejaculation,หลั่งน้ำอสุจิเร็ว,illness,YES,NaN,NaN
3829,NaN,J30,4.0,J304,"Allergic rhinitis, unspecified",เยื่อจมูกอักเสบจากภูมิแพ้ ไม่ระบุรายละเอียด,illness,YES,NaN,NaN
3873,NaN,J40,NaN,J40,"Bronchitis, not specified as acute or chronic",หลอดลมอักเสบ ไม่ระบุว่าเฉียบพลันหรือเรื้อรัง,illness,NO,J400,NaN
4349,NaN,K12,4.0,K1204,Traumatic ulcer (TM),แผลจากการบาดเจ็บ,illness,NO,K120,NaN
4485,NaN,K29,7.0,K297,"Gastritis, unspecified",กระเพาะอาหารอักเสบ ไม่ระบุรายละเอียด,illness,YES,NaN,NaN
4591,NaN,K59,0.0,K590,Constipation,ท้องผูก,illness,YES,NaN,NaN


In [143]:
df[(df['icd10'].isin(filtered_icd10)) & (df['claim_status']=='Decline')]

,created_at,cid,patient,illnes_type,is_follow_up,billing,order_item,total_bill_amount,insurance,response,claim_status,diagnosis,icd10,chief_complaint
5749,2024-08-20 16:10:30.630 +0700,1229901878914,"{""hn"": ""67-09388"", ""dob"": ""2023-06-21"", ""gende...",ILL,N,"[{""billing_initial"": ""0"", ""billing_discount"": ...","[{""initial"": ""0"", ""item_id"": ""99"", ""discount"":...",1500.0,tli,"{""detail"": [{""benefit"": [{""simb_code"": ""1.1.1(...",Decline,"[{""icd10"": ""B974"", ""dx_name"": ""Respiratory syn...",B974,ไข้สูง 1 วันไข้สูง 1 วัน 1 วัน - ไข้สูงไอห่าง...
5978,2024-08-01 17:53:28.239 +0700,1229900890236,"{""hn"": ""66-17273"", ""dob"": ""2000-06-02"", ""gende...",ILL,N,"[{""billing_initial"": ""0"", ""billing_discount"": ...","[{""initial"": ""0"", ""item_id"": ""99"", ""discount"":...",1059.0,tli,"{""detail"": [{""benefit"": [{""simb_code"": ""1.1.1(...",Decline,"[{""icd10"": ""L028"", ""dx_name"": ""Cutaneous absce...",L028,F/U i&d woundF/U i&d wound
5979,2024-08-01 17:58:05.273 +0700,1229900890236,"{""hn"": ""66-17273"", ""dob"": ""2000-06-02"", ""gende...",ILL,N,"[{""billing_initial"": ""0"", ""billing_discount"": ...","[{""initial"": ""0"", ""item_id"": ""99"", ""discount"":...",1059.0,tli,"{""detail"": [{""benefit"": [{""simb_code"": ""1.1.1(...",Decline,"[{""icd10"": ""L02.9"", ""dx_name"": ""Cutaneous absc...",L029,F/U i&d woundF/U i&d wound
9515,2024-08-03 18:14:00.463 +0700,1229900890236,"{""hn"": ""66-17273"", ""dob"": ""2000-06-02"", ""gende...",ILL,N,"[{""billing_initial"": ""0"", ""billing_discount"": ...","[{""initial"": ""0"", ""item_id"": ""99"", ""discount"":...",283.0,tli,"{""detail"": [{""benefit"": [{""simb_code"": ""1.1.1(...",Decline,"[{""icd10"": ""L02.8"", ""dx_name"": ""Cutaneous absc...",L028,ทำแผลต่อเนื่องทำแผลต่อเนื่อง แผลแห้งดีแผลแห้งดี
12307,2024-08-14 15:25:21.122 +0700,1129902711038,"{""hn"": ""65-15147"", ""dob"": ""2021-02-25"", ""gende...",ILL,N,"[{""billing_initial"": ""185"", ""billing_discount""...","[{""initial"": ""185"", ""item_id"": ""NSE"", ""discoun...",1224.0,tli,"{""detail"": [{""benefit"": [{""simb_code"": ""1.1.1(...",Decline,"[{""icd10"": ""B974"", ""dx_name"": ""Respiratory syn...",B974,\r\nนัดตรวจซ้ำหลัง Admit WARD
13668,2024-08-22 18:24:20.950 +0700,1849903097906,"{""hn"": ""1428065"", ""dob"": ""2022-04-15"", ""gender...",ILL,N,"[{""billing_initial"": ""688.00"", ""billing_discou...","[{""initial"": ""425"", ""item_id"": ""S022"", ""discou...",1188.0,tli,"{""detail"": [{""benefit"": [{""simb_code"": ""1.1.1(...",Decline,"[{""icd10"": ""J40"", ""dx_name"": ""Bronchitis, not ...",J40,2วัน ไข้ ไอ อาเจียน น้ำมูกใส
20800,2024-08-24 10:02:25.767 +0700,3860400536843,"{""hn"": ""1507067"", ""dob"": ""1968-04-01"", ""gender...",ILL,N,"[{""billing_initial"": ""420.00"", ""billing_discou...","[{""initial"": ""335"", ""item_id"": ""GE130101"", ""di...",1347.5,tli,"{""detail"": [{""benefit"": [{""simb_code"": ""1.1.1(...",Decline,"[{""icd10"": ""J30.4"", ""dx_name"": ""Allergic rhini...",J304,ไอ มีเสมหะ 1 เดือน
24173,2024-08-03 18:12:28.734 +0700,1102200564519,"{""hn"": ""1003967"", ""dob"": ""2023-01-27"", ""gender...",ILL,N,"[{""billing_initial"": ""120"", ""billing_discount""...","[{""initial"": ""120"", ""item_id"": ""REGFEE1"", ""dis...",2909.0,tli,"{""detail"": [{""benefit"": [{""simb_code"": ""1.1.1(...",Decline,"[{""icd10"": ""A049"", ""dx_name"": ""Bacterial intes...",A049,*
24204,2024-08-24 11:26:29.489 +0700,3100201310639,"{""hn"": ""43-8925"", ""dob"": ""1954-04-14"", ""gender...",ILL,N,"[{""billing_initial"": ""500"", ""billing_discount""...","[{""initial"": ""500"", ""item_id"": ""DF1"", ""discoun...",2485.0,mtl,"{""detail"": [{""benefit"": [{""simb_code"": ""1.2.1(...",Decline,"[{""icd10"": ""J304"", ""dx_name"": ""Allergic rhinit...",J304,คัดจมูก ภูมิแพ้//ขึ้นบัตร
39875,2024-08-17 20:39:08.306 +0700,1103705522334,"{""hn"": ""6215805"", ""dob"": ""2019-08-23"", ""gender...",ILL,N,"[{""billing_initial"": ""600"", ""billing_discount""...","[{""initial"": ""60

In [180]:
import gradio as gr
import pandas as pd
import json

# Mappings for illness type and follow-up status
illness_type_mapping = {
    "ILL": "เจ็บป่วย",
    "ACC": "อุบัติเหตุ",
    "FU": "ผู้ป่วยนอกติดตามอาการหลังจาก Admit เจ็บป่วย",
    "ER": "อุบัติเหตุฉุกเฉินภายใน 24 ชม.",
    "HD": "ผู้ป่วยนอกล้างไต",
    "DEN": "ทันตกรรม"
}

follow_up_mapping = {
    "N": "เข้ารับการรักษาครั้งแรก",
    "Y": "เข้ารักษาแบบต่อเนื่อง"
}

diagnosis_mapping = {
    row['ICD-10 Code']: row['Description'].strip()
    for _, row in disease.iterrows()
}

sample_cids = df[df['is_follow_up']=='N']['cid'].sample(5).tolist()

# Add custom CSS for red borders
custom_css = """
.red-border {
    border: 2px solid red !important;
    border-radius: 5px;
    padding: 5px;
}
"""

# Function to populate patient information based on CID
def populate_patient_info(cid):
    patient_row = df[df["cid"] == cid]
    if not patient_row.empty:
        patient_data = json.loads(patient_row.iloc[0]["patient"])
        icd10_code = patient_row.iloc[0]["icd10"]
        visit_date = patient_row.iloc[0]["created_at"]
        diagnosis_description = diagnosis_mapping.get(icd10_code, "")
        illness_type_mapped = illness_type_mapping.get(patient_row["illnes_type"].iloc[0], "อุบัติเหตุ")
        visit_type_mapped = follow_up_mapping.get(patient_row["is_follow_up"].iloc[0], "Unknown")
        chief_complaint = patient_row["chief_complaint"].iloc[0]

        # Get the billing table and total net
        billing_df, total_net = get_billing_table_with_order_items(cid)
        billing_str = billing_df.to_string(index=False)

        return (
            patient_data.get("title_name", ""),
            patient_data.get("first_name", ""),
            patient_data.get("last_name", ""),
            patient_data.get("hn", ""),
            patient_data.get("dob", ""),
            illness_type_mapped,
            visit_type_mapped,
            chief_complaint,
            diagnosis_description.strip(),
            billing_df,
            total_net
        )
    else:
        return "", "", "", "", "", "", "", "", "", "", 0.0

def populate_sample_data(cid):
    return populate_patient_info(cid)

# Function to get billing table and order items
def get_billing_table_with_order_items(cid):
    patient_row = df[df["cid"] == cid]
    if not patient_row.empty:
        billing_data = json.loads(patient_row.iloc[0]["billing"])
        order_items_data = json.loads(patient_row.iloc[0]["order_item"])

        billing_df = pd.DataFrame(billing_data)
        order_items_df = pd.DataFrame(order_items_data)

        all_rows = []

        for _, billing_row in billing_df.iterrows():
            matching_items = order_items_df[order_items_df['simb_billing_code'] == billing_row['simb_billing_code']]

            # Add a row for the billing category
            all_rows.append({
                "ลำดับ": len(all_rows) + 1,
                "หมวดหมู่ค่าบริการ": f"{billing_row['simb_billing_code']} {billing_row['local_billing_name']}",
                "รายการ": "",
                "จำนวน": billing_row.get('item_amount', ""),  # Use get to safely access the key
                "จำนวนเงิน(ก่อนหักส่วนลด)": float(billing_row['billing_initial']),
                "ส่วนลด": float(billing_row['billing_discount']),
                "จำนวนเงิน(หลังหักส่วนลด)": float(billing_row['billing_net_amount']),
                "Billing Code": "",
                "SIMB": billing_row['simb_billing_code']
            })

            # Add rows for each item under this category
            for _, item in matching_items.iterrows():
                all_rows.append({
                    "ลำดับ": "",
                    "หมวดหมู่ค่าบริการ": item['item_id'],
                    "รายการ": item['item_name'],
                    "จำนวน": item['item_amount'],  # Access item_amount from order_items_df
                    "จำนวนเงิน(ก่อนหักส่วนลด)": float(item['initial']),
                    "ส่วนลด": float(item['discount']),
                    "จำนวนเงิน(หลังหักส่วนลด)": float(item['net_amount']),
                    "Billing Code": billing_row['local_billing_code'],
                    "SIMB": billing_row['simb_billing_code']
                })

        # Summation of numeric columns
        total_initial = billing_df['billing_initial'].astype(float).sum()
        total_discount = billing_df['billing_discount'].astype(float).sum()
        total_net = billing_df['billing_net_amount'].astype(float).sum()

        return pd.DataFrame(all_rows), total_net

    return pd.DataFrame(columns=["ลำดับ", "หมวดหมู่ค่าบริการ", "รายการ", "จำนวน", "จำนวนเงิน(ก่อนหักส่วนลด)", "ส่วนลด", "จำนวนเงิน(หลังหักส่วนลด)", "Billing Code", "SIMB"]), 0.0

# Function to generate text summary for submission
def generate_summary(
    id_card, first_name, last_name, patient_id, visit_date, insurance_detail, billing_str,
    treatment_type, visit_type, chief_complaint, diag_type, diagnosis, total_net
):

    # Format the summary of patient info and insurance details
    patient_info = f"""
    Patient Information:
    - ID Card: {id_card}
    - Name: {title} {first_name} {last_name}
    - Visit Date: {visit_date}
    - Treatment Type: {treatment_type}
    - Visit Type: {visit_type}
    - Chief Complaint: {chief_complaint}
    - Diagnosis: {diagnosis}
    - Billing Info: {billing_str}
    - Total Bill Amount (Net): {total_net} THB
    """

    insurance_info = f"""
    Insurance Details: {insurance_detail}
    """

    return f"{patient_info}\n\n{insurance_info}"

In [184]:
# Gradio UI
with gr.Blocks(css=custom_css) as demo:
    with gr.Tab("Patient and Claim Information"):
        gr.Markdown("## ข้อมูลผู้ป่วยนอก")
        id_card = gr.Textbox(label="เลขบัตรประชาชน *", placeholder="กรอกเลขบัตรประชาชนของผู้ป่วย", elem_classes="red-border")
        sample_cid_buttons = gr.Radio(label="เลือกตัวอย่าง CID", choices=sample_cids, type="value")
        with gr.Row():
            title = gr.Dropdown(
                label="คำนำหน้า",
                choices=["คุณ", "นาง", "นางสาว", "นาย", "Mr", "Ms", "Miss", "Mrs", "เด็กชาย", "เด็กหญิง"]
            )
            first_name = gr.Textbox(label="ชื่อ")
            last_name = gr.Textbox(label="นามสกุล")
        with gr.Row():
            patient_id = gr.Textbox(label="รหัสผู้ป่วย (HN)")
            visit_date = Calendar(label="วันที่เข้ารับการรักษา")

        gr.Markdown("### ข้อมูลส่งเคลม")

        with gr.Row():
            treatment_type = gr.Dropdown(
                label="ประเภทการรักษา",
                value="",
                choices=list(illness_type_mapping.values())
            )
            visit_type = gr.Radio(
                label="ประเภทการเข้ารักษา",
                choices=list(follow_up_mapping.values())
            )
        chief_complaint = gr.Textbox(label="อาการสำคัญที่เข้ามาโรงพยาบาล", value="")

        gr.Markdown("#### Diagnosis Type")
        with gr.Row():
            diag_type = gr.Dropdown(
                label="Diagnosis Type",
                choices=["โรคหลักที่ให้การรักษา (PP)", "โรคประจำตัว (UD)", "การวินิจฉัยร่วม (CM)", "โรคแทรก (CP)", "โรคอื่นๆ (OT)"],
                value='โรคหลักที่ให้การรักษา (PP)'
            )
            diagnosis = gr.Dropdown(
                label="Diagnosis (ICD10) *",
                choices=list(diagnosis_mapping.values()),
                value=""
            )

        gr.Markdown("#### รายการค่ารักษาพยาบาล")
        billing_table = gr.DataFrame(
            headers=["ลำดับ", "หมวดหมู่ค่าบริการ", "รายการ", "จำนวน", "จำนวนเงิน(ก่อนหักส่วนลด)", "ส่วนลด", "จำนวนเงิน(หลังหักส่วนลด)", "Billing Code", "SIMB"],
            interactive=True
        )

    with gr.Tab("Insurance Policy Information"):
        gr.Markdown("## ข้อมูลกรมธรรม์ที่ใช้")
        with gr.Row():
            insurance_detail = gr.Textbox(label="ข้อมูลกรมธรรม์/ความคุ้มครอง", placeholder="กรอกข้อมูลกรมธรรม์ที่ใช้", elem_classes="red-border")

    id_card.change(
        fn=populate_patient_info,
        inputs=[id_card],
        outputs=[id_card, title, first_name, last_name, patient_id, visit_date, treatment_type, visit_type, chief_complaint, diagnosis]
    )

    sample_cid_buttons.change(
        fn=populate_sample_data,
        inputs=[sample_cid_buttons],
        outputs=[id_card, title, first_name, last_name, patient_id, visit_date, treatment_type, visit_type, chief_complaint, diagnosis]
    )

    submit_button = gr.Button("Submit")
    submission_output = gr.Textbox(label="Output Summary", interactive=False)

    submit_button.click(
        fn=generate_summary,
        inputs=[
            id_card, first_name, last_name, patient_id, visit_date, insurance_detail,
            treatment_type, visit_type, chief_complaint, diag_type, diagnosis, billing_table
        ],
        outputs=[submission_output]
    )

# Launch the Gradio interface
demo.launch(debug=True)


/usr/local/lib/python3.10/dist-packages/gradio/components/dropdown.py:226: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include:  or set allow_custom_value=True.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:999: UserWarning: Expected 13 arguments for function <function generate_summary at 0x7ebc99986dd0>, received 12.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:1003: UserWarning: Expected at least 13 arguments for function <function generate_summary at 0x7ebc99986dd0>, received 12.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://50209bc63d31120b46.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:1748: UserWarning: A function (populate_sample_data) returned too many output values (needed: 10, returned: 11). Ignoring extra values.
    Output components:
        [textbox, dropdown, textbox, textbox, textbox, calendar, dropdown, radio, textbox, dropdown]
    Output values returned:
        ["นาย", "ธัชพล", "ยุกตะนันท์", "60-05816", "1981-08-30", "เจ็บป่วย", "เข้ารับการรักษาครั้งแรก", "
นัดมาตรวจซ้ำ...", "",    ลำดับ                                  หมวดหมู่ค่าบริการ  \
0      1   1.1.1(3) Outpatient Medication   ค่ายาผู้ป่วยนอก   
1                                           0110E12S0010L01   
2                                           0102D01A0025L02   
3      4  1.1.12 Nursing and Midwifery Charge   ค่าบริกา...   
4                                                       NSE   
5      6  1.1.14(2) Other Medical Service Charge ค่าบริก...   
6                                                        NH   
7                      

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7862 <> https://50209bc63d31120b46.gradio.live
